
# ***Libraries***

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import numpy as np
from tqdm import tqdm

import gc
import imblearn
from xml.etree import ElementTree
from keras.utils import plot_model

import matplotlib.pyplot as plt
import random
import math
import numpy as np
from numpy import array

import os
from numpy import zeros
from numpy import asarray
import colorsys
import argparse
import imutils
import time

from matplotlib import pyplot
from matplotlib.patches import Rectangle
from keras.models import load_model

from os import listdir
from xml.etree import ElementTree
import json
from xml.dom import minidom

from PIL import Image
import PIL
import glob
from sklearn import metrics

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import tensorflow as tf
import keras
import keras.models as KM
import keras.layers as KL
from keras import backend as K
import tensorflow.python.keras.engine  as ke

from sklearn.metrics import classification_report,confusion_matrix
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

from config import Config
import model as modellib
import visualize
from utils import Dataset
from model import MaskRCNN

import warnings
warnings.filterwarnings('ignore')


In [ ]:
import sys
sys.path.append(os.getcwd()+'/content/drive/MyDrive/Colab_Notebooks/VeDo_mrcnn/mrcnn_vedo')

# ***Functions***

In [ ]:
def TopK(x, k):
    a = dict([(i, j) for i, j in enumerate(x)])
    sorted_a = dict(sorted(a.items(), key = lambda kv:kv[1], reverse=True))
    indices = list(sorted_a.keys())[:k]
    values = list(sorted_a.values())[:k]
    return (indices, values)

In [ ]:
def class_val_func2():
  imgpath = '/content/drive/MyDrive/Colab_Notebooks/Workspace/Objects-Dataset/images/'
  annpath = '/content/drive/MyDrive/Colab_Notebooks/Workspace/Objects-Dataset/annots/'

  n = 0
  mae = 0
  mse = 0
  mses = []

  for filename in listdir(imgpath):
    image_id = filename.split('.')[0]

    n += 1

    img= plt.imread(imgpath + str(image_id) + ".jpg")
    result = mr_model_inf.detect([img])
    r = result[0]

    tree = ET.parse(annpath + str(image_id) +'.xml')
    root = tree.getroot()

    distance = float(root.find('./distance').text)

    y = r['distance'][0]
    mae += abs(y-distance)
    mse += pow(y-distance,2)
    mses.append(pow(y-distance,2))


  return {
      "total": n,
      "mae": mae / n,
      "mse": mse / n,
      "mses": mses
  }


In [ ]:
def custom_acc():
  imgpath = '/content/drive/MyDrive/Colab_Notebooks/Workspace/Objects-Dataset/images/'
  annpath = '/content/drive/MyDrive/Colab_Notebooks/Workspace/Objects-Dataset/annots/'

  n = 0
  s1 = 0
  s2 = 0

  for filename in listdir(imgpath):
    image_id = filename.split('.')[0]

    n += 1

    img= plt.imread(imgpath + str(image_id) + ".jpg")
    result = mr_model_inf.detect([img])
    r = result[0]

    tree = ET.parse(annpath + str(image_id) +'.xml')
    root = tree.getroot()

    distance = float(root.find('./distance').text)

    y = r['distance'][0]
    if abs(y-distance) < 30:
      s1 += 1
    if abs(y-distance) < 15:
      s2 += 1


  return {
      "total": n,
      "s1": s1 / n,
      "s2": s2 / n,
  }


In [ ]:
def class_val_func():
  imgpath = '/content/drive/MyDrive/Colab_Notebooks/Workspace/Objects-Dataset/images/'
  annpath = '/content/drive/MyDrive/Colab_Notebooks/Workspace/Objects-Dataset/annots/'

  n = 0
  mae = 0
  mse = 0

  for filename in listdir(imgpath):
    image_id = filename.split('.')[0]

    n += 1

    img= plt.imread(imgpath + str(image_id) + ".jpg")
    result = mr_model_inf.detect([img])
    r = result[0]

    tree = ET.parse(annpath + str(image_id) +'.xml')
    root = tree.getroot()

    distance = float(root.find('./distance').text)

    y = r['distance'][0]
    mae += abs(y-distance)
    mse += pow(y-distance,2)


  return {
      "total": n,
      "mae": mae / n,
      "mse": mse / n,
  }


In [ ]:
def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return iou

In [ ]:

def evaluate_model(dataset, model):
  imgpath = '/content/drive/MyDrive/Colab_Notebooks/Workspace/Objects-Dataset/images/'
  annpath = '/content/drive/MyDrive/Colab_Notebooks/Workspace/Objects-Dataset/annots/'

  n = 0
  mae = 0
  mse = 0
  accuracy = 0
  class_accuracy = 0
  acc_ious = []
  a_allboxes = []
  p_allboxes = []
  iousum=0

  for image_id in dataset.image_ids:

    img_info = dataset.image_info[image_id]
    img= plt.imread(img_info['path'])
    result = mr_model_inf.detect([img])
    r = result[0]
    pnames = r['class_ids'] - 1
    scores = r['scores']
    n += 1

    tree = ET.parse(img_info['annotation'])
    root = tree.getroot()
    filename = str(root.find('./filename').text)[0:5]

    names = []
    for name in root.findall('./object/name'):
      names.append(int(name.text))

    if ((int(pnames[0])==names[0]) and int(pnames[1])==names[1]) or ((int(pnames[0])==names[1]) and int(pnames[1])==names[0]) :
      class_accuracy +=2

    aboxes = []
    for box in root.findall('.//bndbox'):
      xmin = int(box.find('xmin').text)
      ymin = int(box.find('ymin').text)
      xmax = int(box.find('xmax').text)
      ymax = int(box.find('ymax').text)
      coors = [xmin, ymin, xmax, ymax]
      a_allboxes.append(coors)
      aboxes.append(coors)

    pboxes = r['rois']

    pboxes[0][0],pboxes[0][1] = pboxes[0][1],pboxes[0][0]
    pboxes[0][2],pboxes[0][3] = pboxes[0][3],pboxes[0][2]
    pboxes[1][0],pboxes[1][1] = pboxes[1][1],pboxes[1][0]
    pboxes[1][2],pboxes[1][3] = pboxes[1][3],pboxes[1][2]

    if pboxes[0][0] > pboxes[1][0] and pboxes[0][1] > pboxes[1][1]:
      pboxes[[0,1]] = pboxes[[1,0]]
    p_allboxes.append(pboxes[0])
    p_allboxes.append(pboxes[1])


    iou1 = bb_intersection_over_union(pboxes[0],aboxes[0])
    iou2 = bb_intersection_over_union(pboxes[1],aboxes[1])
    acc_ious.append(iou1)
    acc_ious.append(iou2)


    distance = float(root.find('./distance').text)
    distanceLabel = int(root.find('./distanceLabel').text)

    if distanceLabel == int(r['distance_label']):
      accuracy += 1
    mae += abs(distance- float(r['distance']))
    mse += pow(distance- float(r['distance']),2)

  return {
      "number of samples": n,
      "distance label accuracy": accuracy / n,
      "distance mae": mae/n,
      "distance mse": mse/n,
      "class accuracy": class_accuracy / (2*n),
      "bbox IoU": np.mean(acc_ious)
      }




In [ ]:
plt.rcParams["font.serif"] = "Times New Roman"
csfont = {'fontname':'DejaVu Serif'}
def visualize(imageID):
  # imgpath = '/content/drive/MyDrive/Workspace/Test-Images/'
  # annpath = '/content/drive/MyDrive/Workspace/Test-Images/'
  imgpath = '/content/drive/MyDrive/Colab_Notebooks/Workspace/Objects-Dataset/images/'
  annpath = '/content/drive/MyDrive/Colab_Notebooks/Workspace/Objects-Dataset/annots/'

  img= plt.imread(imgpath + str(imageID) + ".jpg")
  plt.figure(figsize=(5, 5),dpi=300)
  plt.imshow(img)
  result = mr_model_inf.detect([img])
  r = result[0]

  classes = r['class_ids']-1

  # print(classes)

  classNames = ['','']

  if classes[0] ==0:
      classNames[0] = 'balloon'
  elif classes[0] ==1:
      classNames[0] = 'bike'
  elif classes[0] ==2:
      classNames[0] = 'camel'
  elif classes[0] ==3:
      classNames[0] = 'car'
  elif classes[0] ==4:
      classNames[0] = 'helicopter'
  elif classes[0] ==5:
      classNames[0] = 'ladder'
  elif classes[0] ==6:
      classNames[0] = 'sofa'

  if classes[1] ==0:
      classNames[1] = 'balloon'
  elif classes[1] ==1:
      classNames[1] = 'bike'
  elif classes[1] ==2:
      classNames[1] = 'camel'
  elif classes[1] ==3:
      classNames[1] = 'car'
  elif classes[1] ==4:
      classNames[1] = 'helicopter'
  elif classes[1] ==5:
      classNames[1] = 'ladder'
  elif classes[1] ==6:
      classNames[1] = 'sofa'


  tree = ET.parse(annpath + str(imageID) +'.xml')
  root = tree.getroot()

  disLabels = ['Very Close','Close','Far','Too Far']

  distance = float(root.find('./distance').text)
  distanceLabel = int(root.find('./distanceLabel').text)
  # plt.title(f" Actual Label:{disLabels[distanceLabel]},            Predicted Label:{disLabels[r['distance_label']]} \n ActualDistance:{distance:.2f}, PredictedDistance:{r['distance']:.2f}",loc='left', fontsize=9)

  aboxes = []
  for box in root.findall('.//bndbox'):
    xmin = int(box.find('xmin').text)
    ymin = int(box.find('ymin').text)
    xmax = int(box.find('xmax').text)
    ymax = int(box.find('ymax').text)
    coors = [xmin, ymin, xmax, ymax]
    aboxes.append(coors)

  anames = []
  for name in root.findall('.//name'):
    anames.append(name.text)


  ax = plt.gca()
  rects=[]
  for box in aboxes:
    x1, y1, x2, y2 = box
    width, height = x2 - x1, y2 - y1
    rect = Rectangle((x1, y1), width, height, fill=False, color='green')
    rects.append(rect)
    ax.add_patch(rect)
  for i, box in enumerate(r['rois']):
    y1, x1, y2, x2 = box
    width, height = x2 - x1, y2 - y1
    rect = Rectangle((x1, y1), width, height, fill=False, color='red', linestyle='--')
    rects.append(rect)
    ax.annotate(classNames[i], (x1+height - 15, y1-6), color='red', fontsize=5, ha='center', va='center',**csfont)
    ax.add_patch(rect)
  # ax.legend([rects[0],rects[2]], ['ActualBox','PredictedBox'])
  plt.text(5, 440,f" Actual Label:{disLabels[distanceLabel]}, Predicted Label:{disLabels[r['distance_label']]}\n \n ActualDistance:{distance:.2f}, PredictedDistance:{r['distance']:.2f}",\
           bbox=dict(fill=False, linewidth=0), fontsize=6,**csfont )
  plt.axis('off')
  plt.show()

# ***Modeling***

In [ ]:

class myMaskRCNNConfig(Config):
    # give the configuration a recognizable name
    NAME = "MaskRCNN_config"

    # set the number of GPUs to use along with the number of images
    # per GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

    # number of classes (we would normally add +1 for the background)
     # objects + BG
    NUM_CLASSES = 8

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 20

    # Learning rate
    LEARNING_RATE=0.001

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9

    # setting Max ground truth instances
    MAX_GT_INSTANCES=10

    IMAGE_MAX_DIM = 960 #480 * 2

    LOSS_WEIGHTS = {
        "rpn_class_loss": 1.,
        "rpn_bbox_loss": 1.,
        "mrcnn_class_loss": 1.,
        "mrcnn_bbox_loss": 1.,
        "mrcnn_mask_loss": 1,
        "d_output_loss": 1,
        "d_output_lbl_loss": 1,
    }


config= myMaskRCNNConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  960
IMAGE_META_SIZE                20
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [960 960   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1, 'd_output_loss': 1, 'd_output_lbl_loss': 1}
MASK_POOL_SIZE 

In [ ]:

mr_model = modellib.MaskRCNN(mode="training", config=config, model_dir='./')

# ***Load Dataset***

In [ ]:
class Creating_Dataset(Dataset):
    # load the dataset definitions
    def load_dataset(self, dataset_dir, is_train=True):

        # Add classes. We have only one class to add.
        self.add_class("dataset", 0, "0")
        self.add_class("dataset", 1, "1")
        self.add_class("dataset", 2, "2")
        self.add_class("dataset", 3, "3")
        self.add_class("dataset", 4, "4")
        self.add_class("dataset", 5, "5")
        self.add_class("dataset", 6, "6")



        self.dataset_dir = dataset_dir
        # define data locations for images and annotations
        images_dir = dataset_dir + '/images/'
        annotations_dir = dataset_dir + '/annots/'

        # Iterate through all files in the folder to
        #add class, images and annotaions
        for filename in listdir(images_dir):

            # extract image id
            image_id = filename.split('.')[0]

            # large
            if is_train and (int(image_id) % 120) > 90:
                continue
            if not is_train and (int(image_id) % 120) < 90:
                continue

            # setting image file
            img_path = images_dir + filename

            # setting annotations file
            ann_path = annotations_dir + image_id + '.xml'

            # adding images and annotations to dataset
            self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)

    # extract bounding boxes from an annotation file
    def extract_boxes(self, filename):

        # load and parse the file
        tree = ElementTree.parse(filename)
        # get the root of the document
        root = tree.getroot()
        # extract each bounding box
        boxes = list()
        classes = list()
        for box in root.findall('.//object'):
            xmin = int(box.find('.//bndbox/xmin').text)
            ymin = int(box.find('.//bndbox/ymin').text)
            xmax = int(box.find('.//bndbox/xmax').text)
            ymax = int(box.find('.//bndbox/ymax').text)
            class_name = box.find('name').text
            coors = [xmin, ymin, xmax, ymax]
            boxes.append(coors)
            classes.append(class_name)

        # extract image dimensions
        width = int(root.find('.//size/width').text)
        height = int(root.find('.//size/height').text)
        distance = float(root.find('./distance').text)

        return boxes, width, height, classes

    # load the masks for an image
    """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
     """
    def load_mask(self, image_id):
        # get details of image
        info = self.image_info[image_id]

        # define anntation  file location
        path = info['annotation']

        # load XML
        boxes, w, h, classes = self.extract_boxes(path)

        # create one array for all masks, each on a different channel
        masks = zeros([h, w, len(boxes)], dtype='uint8')

        # create masks
        class_ids = list()
        for i in range(len(boxes)):
            box = boxes[i]
            row_s, row_e = box[1], box[3]
            col_s, col_e = box[0], box[2]
            masks[row_s:row_e, col_s:col_e, i] = 1
            class_ids.append(self.class_names.index(classes[i]))
        return masks, asarray(class_ids, dtype='int32')

    # load an image reference
    #Return the path of the image."""
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        print(info)
        return info['path']

In [ ]:
# prepare train set
train_set = Creating_Dataset()
train_set.load_dataset('/content/drive/MyDrive/Colab_Notebooks/Workspace/Objects-Dataset', is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))
# prepare test/val set
test_set = Creating_Dataset()
test_set.load_dataset('/content/drive/MyDrive/Colab_Notebooks/Workspace/Objects-Dataset', is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))

Train: 364
Test: 120


# ***Loading Initial Weights***

In [ ]:
mr_model.load_weights('/content/drive/MyDrive/Colab_Notebooks/Workspace/MRCNN_Modified_Model/base_weights.h5', by_name=True)
mr_model.load_weights('/content/drive/MyDrive/Colab_Notebooks/Workspace/MRCNN_Modified_Model/new_distance_weights.h5', by_name=True)
mr_model.load_weights('/content/drive/MyDrive/Colab_Notebooks/Workspace/MRCNN_Modified_Model/new_cat_weights.h5', by_name=True)

# ***Train Model***

In [ ]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')

print("GPU:", tf.config.list_physical_devices('GPU'))
print("Num GPUs:", len(physical_devices))

In [ ]:
import tensorflow as tf
if tf.test.gpu_device_name():
   print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
   print("Please install GPU version of TF")

Please install GPU version of TF


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


In [ ]:
tf.test.gpu_device_name()

''

In [ ]:
mr_model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=80, layers='heads')

In [ ]:
model_path = '/content/drive/MyDrive/Colab_Notebooks/Workspace/MRCNN-Modified-Model/my_weights3.h5'
mr_model.keras_model.save_weights(model_path)

In [ ]:
mr_model_inf = modellib.MaskRCNN(mode="inference", config=config, model_dir='./')

In [ ]:
mr_model_inf.load_weights(model_path, by_name=True)

In [ ]:
#for test
mr_model_inf.load_weights('/content/drive/MyDrive/Workspace/MRCNN-Modified-Model/base_weights.h5', by_name=True)
mr_model_inf.load_weights('/content/drive/MyDrive/Workspace/MRCNN-Modified-Model/new_distance_weights.h5', by_name=True)
mr_model_inf.load_weights('/content/drive/MyDrive/Workspace/MRCNN-Modified-Model/new_cat_weights.h5', by_name=True)

In [ ]:
plot_model(mr_model.keras_model, '/content/drive/MyDrive/Workspace/MRCNN-Modified-Model/Model\ Diagrams/train.svg', show_shapes=True)
plot_model(mr_model_inf.keras_model, '/content/drive/MyDrive/Workspace/MRCNN-Modified-Model/Model\ Diagrams/inf.svg', show_shapes=True)

# ***Evaluation***

In [ ]:
visualize('00003')

In [ ]:
evaluate_model(test_set, mr_model_inf)